In [ ]:
import pandas as pd
import json
from time import sleep
import requests
import re

In [ ]:
!git clone https://github.com/IlyaGusev/rupo
!cd rupo

Cloning into 'rupo'...
remote: Enumerating objects: 1488, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1488 (delta 0), reused 1 (delta 0), pack-reused 1485
Receiving objects: 100% (1488/1488), 16.38 MiB | 13.22 MiB/s, done.
Resolving deltas: 100% (980/980), done.


In [ ]:
!pip install -r /content/rupo/requirements.txt

  Cloning git://github.com/IlyaGusev/rulm.git (to revision 4e78a49) to /tmp/pip-req-build-olgl8ajk
  Running command git clone -q git://github.com/IlyaGusev/rulm.git /tmp/pip-req-build-olgl8ajk
  Running command git checkout -q 4e78a49
  Cloning git://github.com/IlyaGusev/russ.git (to revision 288fe6a) to /tmp/pip-req-build-_5spqxth
  Running command git clone -q git://github.com/IlyaGusev/russ.git /tmp/pip-req-build-_5spqxth
  Running command git checkout -q 288fe6a
     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 10.5MB 4.5MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 7.6MB 20.9MB/s 
     |████████████████████████████████| 8.2MB 36.3MB/s 
     |████████████████████████████████| 163kB 44.1MB/s 
     |████████████████████████████████| 133kB 43.4MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 122kB 43.9MB/s 
     |████████████████████████

In [ ]:
import sys
sys.path.append('/content/rupo')

In [ ]:
import rupo
%cd rupo
!sh download.sh
from rupo.api import Engine
engine = Engine(language="ru")
engine.load('/content/rupo/rupo/data/stress_models/ru_main', '/content/rupo/rupo/data/dict/zaliznyak.txt')
engine.get_stresses("корова")
from collections import Counter

/content/rupo
--2021-06-07 07:52:46--  https://www.dropbox.com/s/dwkui2xqivzsyw5/generator_model.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dwkui2xqivzsyw5/generator_model.zip [following]
--2021-06-07 07:52:46--  https://www.dropbox.com/s/raw/dwkui2xqivzsyw5/generator_model.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1453e4a6cf685e853bd5f8254c.dl.dropboxusercontent.com/cd/0/inline/BP_D7nfFXt5CggyqF5ZIDcZIYFPMSTEmNlo26VOz1kpVif33_AxxVh1d_HtcKnAnz0Enb7DarY2PzcSxyAwhwZqq6U6aZg7lbmPjtWCYrzHrztUun6SPGMt1OAlmIM2QQN8LtDZY8_SLNQIJnpfKQppQ/file# [following]
--2021-06-07 07:52:47--  https://uc1453e4a6cf685e853bd5f8254c.dl.dropboxusercontent.com/cd/0/inline/BP_D7nfFXt5CggyqF5ZIDcZIYFPMSTEmNlo26VOz1kpVif33_AxxVh1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# собирает инфу о стихах
def take_info(r):
    list_data = []
    # print(r.text.split('ul type="square"'))
    s1 = r.text.split('ul type="square"')[1]
    s2 = r.text.split('ul type="square"')[3].split('</ul>')[0]

    list_d = []
    x = s1.split('<li>') + s2.split('<li>')
    for li in x:
        href = re.findall(r'a href=".+?"', li)
        name = re.findall(r'class="poemlink">.+?</a>', li)
        author = re.findall(r'class="authorlink">.+?</a>', li)
        list_d.append((href, name, author))

    list_d = [x for x in list_d if x != ([], [], [])]
    # print(list_d[1][0][0])
    if len(list_d) < 2:
#         print(list_d)
        return list_data
    else:
        data = '.'.join(list_d[1][0][0].replace('a href="', '').replace('"', '').split('/')[:4])
        for data_str in list_d[1:]:
            link = data_str[0][0].replace('a href="', '').replace('"', '')
            name = data_str[1][0].replace('class="poemlink">', '').replace('</a>', '')
            author = data_str[2][0].replace('class="authorlink">', '').replace('</a>', '')
            list_data.append((link, name, author, data))
#     print(list_data)
    return list_data

In [ ]:
def take_day_give_other_day(r):
    list_other_stixi = r.text.split('Произведения за')[1].split('Произведения  за предыдущий день:')[0]
    list_other_day = r.text.split('Произведения за')[1].split('Произведения  за предыдущий день:')[1]
    others_links = []
    for link in list_other_stixi.split('<a style="text-decoration:none" href="'):
        others_links.append(link.split('"')[0])
    other_day = list_other_day.split('">')[0].replace(' <a style="text-decoration:none" href="', '')
    # data = re.findall(r'Произведения за .+?:</b>', li)[0].replace(':</b>', '').replace('Произведения за ', '')
    return other_day, others_links

In [ ]:
# сделать функцикей
def return_lines_texts(link):
    list_texts = []
    try:
#         print('https://stihi.ru/' + link)
        r = requests.get('https://stihi.ru/' + link) 
        text = r.text.split('<div class="text">')[1].split('</div>')[0].replace('<br>', '')
        list_texts.append(text)
    except IndexError:
        list_texts.append('')
    return list_texts

In [ ]:
# сделатьфунннннцмокова
def split_lines(text_stix, infa):
    text = str(text_stix[0]).replace('&nbsp;', '').replace('&quot;', '').replace('*', '').replace('  ', '').split('\n')
    dict_many_lines_stix = {}
    text = [x for x in text if x != '' and x != ' ']
    dict_many_lines_stix['text_split_n'] = '\n'.join(text)
    
    lines_stix = []
    for line in text:
        lines_stix.append([line, 'all'])
        
    dict_many_lines_stix['stix'] = lines_stix
    dict_many_lines_stix['author'] = infa[2]
    dict_many_lines_stix['data'] = infa[-1]
    dict_many_lines_stix['name'] = infa[1]

    return dict_many_lines_stix


In [ ]:
# ПРОХОДИТСЯ ПО ВСЕЙ СТРОКЕ В ПОИСКАХ ВНУТРЕННЕЙ РИФМЫ. вНУТРЕННЯЯ РИФМА МОЖЕТ БЫТЬ ТОЛЬКО С ОДНИМ ИЗ СЛОВ В СТРОКЕ
def clean_text(text_list):
    text = []
    for i in text_list.split(' '):
        i = i.lower()
        i = i.replace(' -- ', ' ').replace('-', ' ').replace('\t', '').replace('\xa0', '')
        i = re.sub(r"[,!?.;:\(\)\…\'\"\»\«\—\―\–\-0-9]", '', i)
        text.append(i.rstrip(' '))
    return [x for x in text if x !='']



def create_corteg_list_pairs(before, after, meadle_word=False):
    list_edges = []
    if meadle_word!=False:
        for word_b in before:
            list_edges.append((word_b, meadle_word))
            for word_a in after:
                list_edges.append((word_b, word_a))
                list_edges.append((word_a, meadle_word))
    else:
        for word_b in before:
            for word_a in after:
                list_edges.append((word_b, word_a))
    return list_edges



def filter_min_three_letter(words):
#     try:
    
    if len(words[-1]) >= 4:
        pass
    else: 
        words = words[:-2] + [words[-2] + words[-1]]

    words_in_line = []
#     []
    for word in words:
        if is_allowed(word) == True and len(word) >= 4:
#             word = accent.put_stress(word).replace("'", '̀')
            words_in_line.append(word.replace(' ', ''))
    
    if len(words_in_line) >= 3:
        count = len(words_in_line)
        if count%2 == 0:
            before = words_in_line[:int(count/2)]
            after = words_in_line[int(count/2):]
            list_edges = create_corteg_list_pairs(before, after)
            
        elif count%2 == 1:
            before = words_in_line[:int(count/2)]
            after = words_in_line[int(count/2)+1:]
            meadle_word = words_in_line[round(int(count/2))]
            list_edges = create_corteg_list_pairs(before, after, meadle_word)
        
        # list_return = yes_or_no_rhyme(list_edges)
        
        list_return  = []
        for words in list_edges:
            fff = engine.is_rhyme(words[0], words[1])
            if fff == True and words[0] != words[1]:
                list_return.append(words)
        return list_return
    else:
        return []

In [ ]:
#  ФУНКЦИЯ НУЖНА ДЛЯ ОТСЕВА СЛОВ В КОТОРЫХ ВСТРЕЧАЮТСЯ НЕ КИРИЛЛИЧЕСКИЕ СИМВОЛЫ

def is_allowed(string):
    characherRegex = re.compile(r"[^'̀'а-яА-яЁё\s0-9'̀.]")
    string = characherRegex.search(string)
    return not bool(string)

In [ ]:
def find_kind_iambos(text):
  metre = []
  for line in str(text).split('\n'):
    line_stops = []
    for word in clean_text(line):
      # print(engine.get_word_syllables(word))
      line_stops+= engine.get_word_syllables(word)
    if len(line_stops) == 8 or len(line_stops) == 9:
      metre.append('4 stops')
    elif len(line_stops) == 10 or len(line_stops) == 11:
      metre.append('5 stops')
    elif len(line_stops) == 12 or len(line_stops) == 13:
      metre.append('6 stops')
    else:
      metre.append('no')
  count_mostcommon_metre = Counter(metre).most_common(3)
  if len(count_mostcommon_metre) != 1:
    second_value = sum([x[1] for x in count_mostcommon_metre[1:]])
    first_value = count_mostcommon_metre[0][1]
    # print((second_value*100)/(first_value+second_value))
    if (second_value*100)/(first_value+second_value) < 25:
      # print((second_value*100)/(first_value+second_value))
      return count_mostcommon_metre[0][0]
  else:
    # print((second_value*100)/(first_value+second_value))
    return count_mostcommon_metre[0][0]
  # if len(count_mostcommon_metre) == 1 and count_mostcommon_metre[0][0] != 'no':
    # print(count_mostcommon_metre)
  

In [ ]:
def ces(text1, stops):
  # print(stops)
  if stops == '6 stops':
    s = 6
  elif stops == '5 stops':
    s = 4
  else:
    return False
  result = []
  text = text1.split('\n')
  for line in text:
    len_syll = 0
    for word in clean_text(line):
      len_syll += len(engine.get_word_syllables(word))
      if len_syll == s:
        result.append('next_line')
        break
      elif len_syll >= s:
        result.append('stop')
        break

  count_mostcommon_ces = Counter(result)
  if len(count_mostcommon_ces) != 1:
    second_value = count_mostcommon_ces['stop']
    first_value = count_mostcommon_ces['next_line']
    if (second_value*100)/(first_value+second_value) <= 25:
      return True
  elif count_mostcommon_ces['next_line'] != 0:
    return True

In [ ]:
def filter_stix(doc):

  if len(doc['stix']) >= 4:
    if len(doc['stix'][0][0]) < 70 and len(doc['stix'][1][0]) < 70 and len(doc['stix'][2][0]) < 70 and len(doc['stix'][3][0]) < 70:
      text = doc['text_split_n']
      metre = engine.classify_metre(text)
      if metre == 'iambos' or metre == 'choreios':
        doc['metre'] = metre
        # print(metre, 'metre')
        

        # проверка регулярности цезуры и регулярности
        stops = find_kind_iambos(doc['text_split_n'])
        # print(stops, 'stops')
        if ces(doc['text_split_n'], stops) == True:
          doc['stops'] = stops

          # ищет пары слов в строке в которых будет потом искаться рифма
          for index_line, text in enumerate(doc['stix']):
            words = clean_text(text[0])
    #         print(words)
            if len(words) >= 4 and words!= [] and len(words) <= 12:
                rhyme_words = filter_min_three_letter(words)
                if rhyme_words!=[]:
                  doc['yes'] = rhyme_words
                  doc['stix'][index_line][-1] = 'yes'
                  with open('/content/drive/MyDrive/new_data.json', 'a', encoding = 'utf-8') as f:
                    json.dump(doc, f, ensure_ascii=False)
                    f.write('\n')




In [1]:
# чтение
# with open('/content/drive/MyDrive/new_data.json', 'r') as f:
#   file = f.read().split('\n')
#   for d in file[:-1]:
#     do = json.loads(d)
#     print(do)

In [ ]:
all_info = []

count = 13

r = requests.get('https://stihi.ru/poems/list.html?day=27&month=08&year=2020&topic=all')
print('!')

corteges_infos = take_info(r)
for stix_info in corteges_infos:
    text_stix = return_lines_texts(stix_info[0])
    dict_to_DB = split_lines(text_stix, stix_info)
other_day, others_links = take_day_give_other_day(r)


while count > 0:
    for link in others_links[1:]:
        # sleep(3)
        print(link)
        r = requests.get('https://stihi.ru' + link)
        corteges_infos = take_info(r)
        for stix_info in corteges_infos:
            text_stix = return_lines_texts(stix_info[0])
            doc = split_lines(text_stix, stix_info)

            # фильтрация
            filter_stix(doc)
            # insert_document_one(collection_3, doc)
            

                          
            
    r = requests.get('https://stihi.ru' + other_day)
    other_day, others_links = take_day_give_other_day(r)
    count -= 1 
    print(count, 'https://stihi.ru' + other_day)

!
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8951
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8921
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8891
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8861
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8831
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8801
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8771
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8741
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8711
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8681
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8651
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8621
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8591
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8561
/poems/list.html?day=27&month=08&year=2020&topic=all&start=8531
/poems/list.html?day=27&month=08&year=

KeyboardInterrupt: ignored